In [ ]:
from PIL import Image
import os
import numpy as np
import random 
import matplotlib.pyplot as plt 
import scipy.misc
import cv2
import copy
import time

# data path 
depth_data_path = "/mnt/data/jarvislam1999/data/subject01_depth/rgb/"
background_data_path = "/mnt/data/jarvislam1999/data/general_bg/"
foreground_data_path = "/mnt/data/jarvislam1999/data/subject01_nobg/rgb/"

# store path
added_bg_data_path = "/mnt/data/jarvislam1999/data/"

#start_time = time.time()

# get all subdirectories that contain images (frames)
#background_data_subdirt = os.listdir(background_data_path)
depth_data_subdirt = [r[0] for r in os.walk(depth_data_path)]
background_data_subdirt = [r[0] for r in os.walk(background_data_path)]
foreground_data_subdirt = [r[0] for r in os.walk(foreground_data_path)]
depth_data_subdirt.sort()
foreground_data_subdirt.sort()

background_data_subdirt = background_data_subdirt[1:]
background_data_subdirt = random.sample(background_data_subdirt, 100)
background_data_subdirt.sort()
foreground_data_subdirt = foreground_data_subdirt[1:]
fg_frames = os.listdir(foreground_data_subdirt[0])
fg_frames.sort()
depth_data_subdirt = foreground_data_subdirt[0].replace(foreground_data_path + '/', depth_data_path)
depth_frames = depth_data_subdirt + '/' + fg_frames[0]
#print("--- Finding and sorting dir: %s seconds ---" % (time.time() - start_time))
iter_bg = 0
# loop through all subdirts (a subdirt contains the frames of one gesture)
for subdirt_bg in background_data_subdirt:
    #if (iter_bg == 2):
    #    iter_bg += 1
    #    continue
    '''
    if (#subdirt_bg != '/mnt/data/jarvislam1999/data/subject01_bg/castle' and \
       #subdirt_bg != '/mnt/data/jarvislam1999/data/subject01_bg/hotel' and \
       subdirt_bg != '/mnt/data/jarvislam1999/data/subject01_bg/moving_crerar' and \
       subdirt_bg != '/mnt/data/jarvislam1999/data/subject01_bg/office01' and \
       subdirt_bg != '/mnt/data/jarvislam1999/data/subject01_bg/office05' and \
       subdirt_bg != '/mnt/data/jarvislam1999/data/subject01_bg/still_crerar'):
        #iter_bg += 1
        continue
    '''    
     
    #if (iter_bg <= 7 or iter_bg >= 12):
    #    iter_bg += 1
    #    continue
    iter_bg += 1
    bg_frames = os.listdir(subdirt_bg)
    bg_frames.sort()
    bg_frames1 = copy.deepcopy(bg_frames)
    
    iter_fg = 0
    for subdirt_fg in foreground_data_subdirt:
        #start_time = time.time()
        
        #if (iter_fg == 4):
        #    continue
        #iter_fg += 1
        #print(subdirt_fg)
        #if (iter_fg <= 172):
        #    iter_fg += 1
        #    continue
        iter_fg += 1
        subdirt_depth = subdirt_fg.replace(foreground_data_path, depth_data_path)
        #print(subdirt_depth)
        if not os.path.isdir(subdirt_depth):
            #print('Wop')
            continue
        
        fg_frames = os.listdir(subdirt_fg)
        fg_frames.sort()
        #print(fg_frames)
        
        # Making sure that the bkg and frg frames match in length
        if (len(bg_frames) == 1):
            bg_frames *= len(fg_frames)
        if (len(bg_frames) < len(fg_frames)):
            bg_frames += [bg_frames[len(bg_frames) - 1] * (len(fg_frames) - len(bg_frames))]
        if ((len(bg_frames1) > len(fg_frames)) and (len(bg_frames1) >= (len(foreground_data_subdirt)* len(fg_frames)))):
            print(iter_fg -1)
            bg_frames = bg_frames1[(iter_fg-1)*10:(iter_fg-1)*10+10]
            #print(bg_frames)
        if ((len(bg_frames1) > len(fg_frames)) and (len(bg_frames1) < (len(foreground_data_subdirt)* len(fg_frames)))):
            #print('Yo')
            bg_frames = bg_frames1[0:10]
        #print(bg_frames)
        frames = []
        for i in range(0, len(fg_frames)):
            frames.append([bg_frames[i], fg_frames[i]])
        
        #print("--- Preparing frames: %s seconds ---" % (time.time() - start_time))
        
        # loop through each frame in a gesture
        for frame in frames:
            # load the pixels of a background image (a frame)
            bg_path = subdirt_bg+'/'+frame[0]
            bg_img = Image.open(bg_path, 'r')
            pixels_of_bg_img = np.array(bg_img.getdata())
            pixels_of_bg_img = np.reshape(pixels_of_bg_img, (480, 640, 3))
            pixels_bg_img = copy.deepcopy(pixels_of_bg_img)
            #plt.imshow(pixels_of_bg_img, interpolation="none")
            #plt.show()
        
            # load the pixels of an foreground image (a frame)
            fg_path = subdirt_fg+'/'+frame[1]
        
            fg_img = Image.open(fg_path, 'r')
            pixels_of_fg_img = np.array(fg_img.getdata())
            pixels_of_fg_img = np.reshape(pixels_of_fg_img, (480, 640, 3))
            #plt.imshow(pixels_of_fg_img, interpolation="none")
            #plt.show()
            
            # load the pixels of a depth image ( a frame)
            depth_path = subdirt_depth + '/' + frame[1]
            if not os.path.isfile(depth_path):
                continue
            
            depth_img = Image.open(depth_path, 'r')
            pixels_of_depth_img = np.array(depth_img.getdata())
            pixels_of_depth_img = np.reshape(pixels_of_depth_img, (480, 640, 3))
            #plt.imshow(pixels_of_depth_img, interpolation="none")
            #plt.show()
            name = "subject01_setting1_01_syn_bg"
            
            if (os.path.isfile(added_bg_data_path+name+'_mask'+'/rgb/'+fg_path.replace(foreground_data_path, "")+'.npy')):
                temp_matrix_1 = np.load(added_bg_data_path+name+'_mask'+'/rgb/'+fg_path.replace(foreground_data_path, "")+'.npy')
            else:
                #print('Going through shade removal')
                #temp_matrix_1 = (pixels_of_fg_img==[0, 0, 0])
                #temp_matrix_1 = np.greater([2,2,2], pixels_of_fg_img)
                temp_matrix_1 = np.logical_and(np.greater([35,35,35], pixels_of_depth_img), \
                                              np.greater([5,5,5], pixels_of_fg_img))
                #temp_matrix_1 = np.logical_and(np.greater([40,40,40], pixels_of_depth_img), \
                #                               pixels_of_fg_img==[0, 0, 0])
                
                # Applying shade removal code
                temp_matrix_1 = np.array(temp_matrix_1)
                temp_matrix_1 = np.logical_not(temp_matrix_1)
                kernel = np.ones((10,10), np.uint8)
                temp_matrix_1 = temp_matrix_1.astype('uint8')
                temp_matrix_1 = cv2.morphologyEx(temp_matrix_1, cv2.MORPH_OPEN, kernel)
                temp_matrix_1 = np.logical_not(temp_matrix_1)
                
                
                # Making all r,g,b channel agree
                temp_matrix_r = np.logical_and(temp_matrix_1[:,:,0], temp_matrix_1[:,:,1])
                temp_matrix_r = np.logical_and(temp_matrix_r, temp_matrix_1[:,:,2])
                temp_matrix_1[:,:,0] = temp_matrix_r
                temp_matrix_1[:,:,1] = temp_matrix_r
                temp_matrix_1[:,:,2] = temp_matrix_r
                #print(temp_matrix_1)
            
            # Taking out background
            pixels_of_bg_img = pixels_of_bg_img * temp_matrix_1
            #plt.imshow(pixels_of_bg_img, interpolation="none")
            #plt.show()
            
            pixels_added_bg_img = pixels_of_bg_img + pixels_of_fg_img
            pixels_added_bg_img = np.array(pixels_added_bg_img, dtype=np.uint8)
            #plt.imshow(pixels_added_bg_img, interpolation="none")
            #plt.show()
            
            # pixels_added_bg_img = cv2.GaussianBlur(pixels_added_bg_img, (3,3), 0)
            #plt.imshow(pixels_added_bg_img, interpolation="none")
            #plt.show()
            
            added_bg_img = Image.fromarray(pixels_added_bg_img)
            
            number = ("000%d" % iter_bg if (iter_bg <= 9) \
            else ("00%d" % iter_bg if (iter_bg <= 999) else ("%d" %iter_bg) )) 
    
            if not os.path.isdir(added_bg_data_path+name+ number):
                print("create new dir: " + added_bg_data_path+name+ number)
                os.mkdir(added_bg_data_path+name+ number)
                os.mkdir(added_bg_data_path+name+ number+'/rgb/')
            if not os.path.isdir(added_bg_data_path+name+ number+'/rgb/'+subdirt_fg.replace(foreground_data_path, "")):
                print("create new dir: " + added_bg_data_path+name+ number+'/rgb/'+subdirt_fg.replace(foreground_data_path, ""))                 
                os.mkdir(added_bg_data_path+name+ number+'/rgb/'+subdirt_fg.replace(foreground_data_path, ""))
            added_bg_img.save(added_bg_data_path+name+ number+'/rgb/'+fg_path.replace(foreground_data_path, ""))
            
            if not os.path.isdir(added_bg_data_path+name+'_mask'):
                print("create new dir: " + added_bg_data_path+name+'_mask')
                os.mkdir(added_bg_data_path+name+'_mask')
                os.mkdir(added_bg_data_path+name+'_mask'+'/rgb/')
            if not os.path.isdir(added_bg_data_path+name+'_mask'+'/rgb/'+subdirt_fg.replace(foreground_data_path, "")):
                print("create new dir: " + added_bg_data_path+name+'_mask'+'/rgb/'+subdirt_fg.replace(foreground_data_path, ""))                 
                os.mkdir(added_bg_data_path+name+'_mask'+'/rgb/'+subdirt_fg.replace(foreground_data_path, ""))
            np.save(added_bg_data_path+name+'_mask'+'/rgb/'+fg_path.replace(foreground_data_path, ""), temp_matrix_1)

create new dir: /mnt/data/jarvislam1999/data/subject01_setting1_01_syn_bg0001/rgb/quick_pronation_05_all
create new dir: /mnt/data/jarvislam1999/data/subject01_setting1_01_syn_bg_mask/rgb/quick_pronation_05_all
create new dir: /mnt/data/jarvislam1999/data/subject01_setting1_01_syn_bg0001/rgb/quick_pronation_06_all
create new dir: /mnt/data/jarvislam1999/data/subject01_setting1_01_syn_bg_mask/rgb/quick_pronation_06_all
create new dir: /mnt/data/jarvislam1999/data/subject01_setting1_01_syn_bg0001/rgb/quick_pronation_07_all
create new dir: /mnt/data/jarvislam1999/data/subject01_setting1_01_syn_bg_mask/rgb/quick_pronation_07_all
create new dir: /mnt/data/jarvislam1999/data/subject01_setting1_01_syn_bg0001/rgb/quick_pronation_08_all
create new dir: /mnt/data/jarvislam1999/data/subject01_setting1_01_syn_bg_mask/rgb/quick_pronation_08_all
create new dir: /mnt/data/jarvislam1999/data/subject01_setting1_01_syn_bg0001/rgb/quick_pronation_09_all
create new dir: /mnt/data/jarvislam1999/data/subjec

create new dir: /mnt/data/jarvislam1999/data/subject01_setting1_01_syn_bg0001/rgb/quick_pronation_41_all
create new dir: /mnt/data/jarvislam1999/data/subject01_setting1_01_syn_bg_mask/rgb/quick_pronation_41_all
create new dir: /mnt/data/jarvislam1999/data/subject01_setting1_01_syn_bg0001/rgb/quick_pronation_42_all
create new dir: /mnt/data/jarvislam1999/data/subject01_setting1_01_syn_bg_mask/rgb/quick_pronation_42_all
create new dir: /mnt/data/jarvislam1999/data/subject01_setting1_01_syn_bg0001/rgb/quick_pronation_43_all
create new dir: /mnt/data/jarvislam1999/data/subject01_setting1_01_syn_bg_mask/rgb/quick_pronation_43_all
create new dir: /mnt/data/jarvislam1999/data/subject01_setting1_01_syn_bg0001/rgb/quick_pronation_44_all
create new dir: /mnt/data/jarvislam1999/data/subject01_setting1_01_syn_bg_mask/rgb/quick_pronation_44_all
create new dir: /mnt/data/jarvislam1999/data/subject01_setting1_01_syn_bg0001/rgb/quick_pronation_45_all
create new dir: /mnt/data/jarvislam1999/data/subjec

create new dir: /mnt/data/jarvislam1999/data/subject01_setting1_01_syn_bg0001/rgb/quick_pronation_80_all
create new dir: /mnt/data/jarvislam1999/data/subject01_setting1_01_syn_bg_mask/rgb/quick_pronation_80_all
create new dir: /mnt/data/jarvislam1999/data/subject01_setting1_01_syn_bg0001/rgb/quick_pronation_81_all
create new dir: /mnt/data/jarvislam1999/data/subject01_setting1_01_syn_bg_mask/rgb/quick_pronation_81_all
create new dir: /mnt/data/jarvislam1999/data/subject01_setting1_01_syn_bg0001/rgb/quick_pronation_82_all
create new dir: /mnt/data/jarvislam1999/data/subject01_setting1_01_syn_bg_mask/rgb/quick_pronation_82_all
create new dir: /mnt/data/jarvislam1999/data/subject01_setting1_01_syn_bg0001/rgb/quick_pronation_83_all
create new dir: /mnt/data/jarvislam1999/data/subject01_setting1_01_syn_bg_mask/rgb/quick_pronation_83_all
create new dir: /mnt/data/jarvislam1999/data/subject01_setting1_01_syn_bg0001/rgb/quick_pronation_84_all
create new dir: /mnt/data/jarvislam1999/data/subjec

In [2]:
background_data_subdirt.sort()

['/mnt/data/jarvislam1999/data/subject01_bg/capitol',
 '/mnt/data/jarvislam1999/data/subject01_bg/castle',
 '/mnt/data/jarvislam1999/data/subject01_bg/crerar_intern',
 '/mnt/data/jarvislam1999/data/subject01_bg/hotel',
 '/mnt/data/jarvislam1999/data/subject01_bg/moving_crerar',
 '/mnt/data/jarvislam1999/data/subject01_bg/night',
 '/mnt/data/jarvislam1999/data/subject01_bg/office01',
 '/mnt/data/jarvislam1999/data/subject01_bg/office02',
 '/mnt/data/jarvislam1999/data/subject01_bg/office03',
 '/mnt/data/jarvislam1999/data/subject01_bg/office04',
 '/mnt/data/jarvislam1999/data/subject01_bg/office05',
 '/mnt/data/jarvislam1999/data/subject01_bg/office06',
 '/mnt/data/jarvislam1999/data/subject01_bg/office07',
 '/mnt/data/jarvislam1999/data/subject01_bg/oildrill',
 '/mnt/data/jarvislam1999/data/subject01_bg/still_crerar',
 '/mnt/data/jarvislam1999/data/subject01_bg/waterfall']

In [5]:
background_data_subdirt[3:8]

['/mnt/data/jarvislam1999/data/subject01_bg/hotel',
 '/mnt/data/jarvislam1999/data/subject01_bg/moving_crerar',
 '/mnt/data/jarvislam1999/data/subject01_bg/night',
 '/mnt/data/jarvislam1999/data/subject01_bg/office01',
 '/mnt/data/jarvislam1999/data/subject01_bg/office02']

In [ ]:
foreground_data_subdirt[0]

In [2]:
pwd


'/mnt/data/jarvislam1999/Real-time-GesRec/annotation_ems'

In [27]:
depth_frames

'/mnt/data/jarvislam1999/data/subject04_nobg/rgb/quick_pronation_01_all/00001.jpg'

In [32]:
foreground_data_subdirt[0].replace(foreground_data_path, depth_data_path)

'/mnt/data/jarvislam1999/data/subject04_depth/rgb/quick_pronation_01_all'

In [29]:
foreground_data_subdirt[0]

'/mnt/data/jarvislam1999/data/subject04_nobg/rgb/quick_pronation_01_all'